In [74]:
import numpy as np  
import os 
from matplotlib import pyplot as plt 
import math

currentDir = os.getcwd()
firstPath = os.path.join(currentDir,"dataFirstMethod.txt")
secondPath = os.path.join(currentDir,'dataSecondMethod.txt')

In [75]:
def convertNumbersRA(inputString):
    prescalers = [15,15/60,15/3600] # Make them degrees, arcminutes,arcseconds
    finalDegree = 0
    for time, number in zip(inputString,prescalers):
        finalDegree += float(time)*number
    return finalDegree

def convertNumbersDELTA(inputString):
    prescalers = [1,1/60,1/3600] # Make them degrees, arcminutes,arcseconds
    finalDegree = 0
    for time, number in zip(inputString,prescalers):
        finalDegree += float(time)*number
    return finalDegree

In [76]:
firstMethodData = {"NAME":[],"RA":[],"DELTA":[]}
secondMethodData = {
    'PARALLAX' : [],
    'PARALLAX_ERROR' : [],
    'VELOCITY' : [],
    'VELOCITY_ERROR' : [],
    'MU_A_COS(DELTA)' : [],
    'MU_A_COS(DELTA)_ERROR' : [],
    'MU_DELTA' : [],
    'MU_DELTA_ERROR' : []
}
keysArray = ['PARALLAX',
    'PARALLAX_ERROR',
    'VELOCITY',
    'VELOCITY_ERROR', 
    'MU_A_COS(DELTA)',
    'MU_A_COS(DELTA)_ERROR',
    'MU_DELTA' ,
    'MU_DELTA_ERROR']

with open(firstPath, 'r') as file:
    line = file.readline()
    while line:
        currentLine = line.strip("\n").split(" ")
        RAString, DELTAString = currentLine[1:4] , currentLine[4:7] #Make all the angles into degrees
        firstMethodData["NAME"].append(currentLine[0])
        firstMethodData["RA"].append(convertNumbersRA(RAString))
        firstMethodData["DELTA"].append(convertNumbersDELTA(DELTAString))
        line = file.readline()
        
with open(secondPath, 'r') as file:
    for line in file:
        currentLine = line.strip("\n").split(" ")[1::] #Clear the line by ignoring the first elements that correspond to the name
        #print(currentLine)
        for index, value in enumerate(currentLine):
            trueValue,trueError = value.split('±')
            keyIdValue,keyIdError = keysArray[index*2],keysArray[index*2+1]
            secondMethodData[keyIdValue].append(float(trueValue))
            secondMethodData[keyIdError].append(float(trueError))
            
for keyToPop in keysArray[2::]:
    firstMethodData[keyToPop] = secondMethodData.pop(keyToPop)
secondMethodData['NAME'] = firstMethodData['NAME']


In [77]:
#The formula for finding the difference between a star (alpha,delta) and the convergent point(alpha_conv,delta_conv) is:
#cos(theta) = sin(delta)*sin(delta_c)+cos(delta)*cos(delta_c)*cos(alpha-alpha_conv)
def findAngleTheta(alpha,delta): #Takes as arguments the RA and DELTA values of the dict row and returns the assigned THETA angle in degrees
    alpha_conv,delta_conv = 96.6, 5.8
    degreeToRadians = math.pi/180
    part1 = math.sin(delta*degreeToRadians)*math.sin(delta_conv*degreeToRadians)
    part2 = math.cos(delta*degreeToRadians)*math.cos(delta_conv*degreeToRadians)*math.cos((alpha-alpha_conv)*degreeToRadians)
    return math.acos(part1 + part2)/degreeToRadians

def addThetaAngleColumn(dictionary): #Takes the dictionary of data and returns the array of all the THETA angles in degrees
    if 'THETA' in dictionary:
        return dictionary['THETA']
    deltaArray,alphaArray = dictionary['DELTA'],dictionary['RA']
    thetaArray = []
    for alpha,delta in zip(alphaArray,deltaArray):
        thetaArray.append(findAngleTheta(alpha,delta))
    return thetaArray

def addTheMuColumn(dictionary):
    if 'MU' in dictionary:
        return dictionary['MU']
    MU_DELTA, MU_A_COS = dictionary['MU_DELTA'], dictionary['MU_A_COS(DELTA)']
    MuArray = []
    for muDelta,muA in zip(MU_DELTA,MU_A_COS):
        MuArray.append(math.sqrt(muDelta**2 + muA**2))
    return MuArray   

def addTheDistanceColumn(dictionary):
    if 'DISTANCE' in dictionary:
        return dictionary['DISTANCE']
    thetaArray,muArray,velocityArray = dictionary['THETA'], dictionary['MU'], dictionary['VELOCITY']
    degreeToRadians = math.pi/180
    distanceArray = []
    for theta,mu,velocity in zip(thetaArray,muArray,velocityArray):
        distanceArray.append(1000*velocity*math.tan(theta*degreeToRadians)/(4.74*mu))
    return distanceArray

def addTheErrorColumn(dictionary):
    if 'ERROR_DISTANCE' in dictionary:
        return dictionary['ERROR_DISTANCE']
    xArray,errorxArray,yArray,erroryArray,muArray = dictionary['MU_A_COS(DELTA)'],dictionary['MU_A_COS(DELTA)_ERROR'],dictionary['MU_DELTA'],dictionary['MU_DELTA_ERROR'],dictionary['MU']
    muErrorArray,distanceErrorArray = [], []
    for x,errx,y,erry,mu in zip(xArray,errorxArray,yArray,erroryArray,muArray):
        muErrorArray.append(math.sqrt((x*errx)**2+(y*erry)**2)/mu)
    velArray,distanceArray,velocityErrorArray = dictionary['VELOCITY'], dictionary['DISTANCE'],dictionary['VELOCITY_ERROR']
    for vel,velerr,mu,muerr,r in zip(velArray,velocityErrorArray,muArray,muErrorArray,distanceArray):
        distanceErrorArray.append(r*math.sqrt((velerr/vel)**2+(muerr/mu)**2))
    return distanceErrorArray
    

In [78]:
firstMethodData.update({'THETA': addThetaAngleColumn(firstMethodData)})
firstMethodData.update({'MU': addTheMuColumn(firstMethodData)})
firstMethodData.update({'DISTANCE': addTheDistanceColumn(firstMethodData)})
firstMethodData.update({'DISTANCE_ERROR':addTheErrorColumn(firstMethodData)})

In [79]:
def addSecondMethodDistance(dictionary): 
    if 'DISTANCE' in dictionary:
        return dictionary['DISTANCE']
    distanceArray,parallaxArray = [], dictionary['PARALLAX']
    for par in parallaxArray:
        distanceArray.append(1000/par)
    return distanceArray  

def addSecondMethodDistanceError(dictionary):
    if 'DISTANCE_ERROR' in dictionary:
        return dictionary['DISTANCE_ERROR']
    parallaxArray, parallaxErrorArray,distanceArray = dictionary['PARALLAX'],dictionary['PARALLAX_ERROR'],dictionary['DISTANCE']
    distanceErrorArray = []
    for par,parerror,dis in zip(parallaxArray, parallaxErrorArray,distanceArray):
        distanceErrorArray.append(par*parerror/dis)
    return distanceErrorArray
    

In [80]:
secondMethodData.update({'DISTANCE': addSecondMethodDistance(secondMethodData)})
secondMethodData.update({'DISTANCE_ERROR': addSecondMethodDistanceError(secondMethodData)})

In [81]:
def clearData(dictionaryFirstMethod,dictionarySecondMethod):
    threshold,center = 10 ,46.34
    totalData = {
        'NAME':[],
        'DISTANCE_FIRST':[],
        'DISTANCE_FIRST_ERROR':[],
        'DISTANCE_SECOND':[],
        'DISTANCE_SECOND_ERROR' : []
        }
    distArray1,distArray2 = dictionaryFirstMethod['DISTANCE'],dictionarySecondMethod['DISTANCE']
    for index, (starDist,starDist2) in enumerate(zip(distArray1,distArray2)):
        if (abs(starDist - center) < threshold) and (abs(starDist2 - center) < threshold): # we will require that by both methods the stars are in the center, as otherwise it will unwise
            totalData['NAME'].append(dictionaryFirstMethod['NAME'][index])
            totalData['DISTANCE_FIRST'].append(dictionaryFirstMethod['DISTANCE'][index])
            totalData['DISTANCE_FIRST_ERROR'].append(dictionaryFirstMethod['DISTANCE_ERROR'][index])
            totalData['DISTANCE_SECOND'].append(dictionarySecondMethod['DISTANCE'][index])
            totalData['DISTANCE_SECOND_ERROR'].append(dictionarySecondMethod['DISTANCE_ERROR'][index])
    return totalData

def addDifferenceColumn(dictionary):
    if 'DIFFERENCE' in dictionary:
        return dictionary
    differenceArray, diffErrorArray = [], []
    dis1Array,dis2Array, err1Array, err2Array = dictionary['DISTANCE_FIRST'],dictionary['DISTANCE_SECOND'],dictionary['DISTANCE_FIRST_ERROR'],dictionary['DISTANCE_SECOND_ERROR']
    for dis1,dis2,err1,err2 in zip(dis1Array,dis2Array, err1Array, err2Array):
        differenceArray.append(dis1-dis2)
        diffErrorArray.append(err1+err2)
    return {'DIFFERENCE': differenceArray,'DIFFERENCE_ERROR': diffErrorArray}

def calculateAverage(valueArray,valueErrorArray,name):
    valueAverage, valueErrorAverage, size = 0, 0, len(valueArray)
    for val,err in zip(valueArray,valueErrorArray):
        valueAverage += val 
        valueErrorAverage += err
    print(f'The mean distance to the cluster using the '+ name +f" method is {valueAverage/size} ± {valueErrorAverage/size} pc")
    pass
    # return valueAverage/size, valueErrorAverage/size

In [82]:
totalData = clearData(firstMethodData,secondMethodData)
totalData.update(addDifferenceColumn(totalData))

In [83]:
totalData

{'NAME': ['14838',
  '18170',
  '18735',
  '20205',
  '20261',
  '20400',
  '20455',
  '20542',
  '20635',
  '20711',
  '20713',
  '20842',
  '20885',
  '20889',
  '20894',
  '20901',
  '21029',
  '21036',
  '21039',
  '21137',
  '21152',
  '21459',
  '21589',
  '21683',
  '22044',
  '22203',
  '23497',
  '23983'],
 'DISTANCE_FIRST': [39.107610614413005,
  40.9020048222674,
  38.854267136834224,
  44.94890902880092,
  43.507543790591804,
  42.412913068256664,
  47.51629744857027,
  45.06187039141562,
  47.21723002046411,
  42.26942382824494,
  43.373743222239035,
  47.55534036219393,
  47.37529356118704,
  45.12621729544459,
  43.341932902733035,
  45.48097529589755,
  46.66856162612917,
  42.287867286550224,
  45.14830119710423,
  38.74319014775086,
  39.78641571816683,
  46.14636094537459,
  47.795566368239356,
  47.430246740979385,
  40.403986890677466,
  46.82086590534784,
  47.93547440924264,
  51.54074093098507],
 'DISTANCE_FIRST_ERROR': [0.8381785113976163,
  2.93618361457066,
 

In [84]:
calculateAverage(totalData["DISTANCE_FIRST"],totalData['DISTANCE_FIRST_ERROR'],'first')
calculateAverage(totalData["DISTANCE_SECOND"],totalData['DISTANCE_SECOND_ERROR'],'second')

The mean distance to the cluster using the first method is 44.45568396271792 ± 1.8275206846673697 pc
The mean distance to the cluster using the second method is 47.02007207663225 ± 0.41327852139285726 pc
